In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
DESCRIPTION 

Compute statistics for mean climatology vs. blocking climatology.

"""

# Import modules
import numpy as np
import netCDF4
import pandas as pd

# Define path
path = '/Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/'

In [2]:
# Import data
mod = netCDF4.Dataset(path + 'final_climatologies.nc')
block_se_sw = netCDF4.Dataset(path + 'myd06_radiative_flux_blocking/MYD06_SW_Fluxes_SE.nc')
block_se_lw = netCDF4.Dataset(path + 'myd06_radiative_flux_blocking/MYD06_LW_Fluxes_SE.nc')

# SW
sw_sw = netCDF4.Dataset(path + 'myd06_radiative_flux_blocking/MYD06_SW_Fluxes_SW.nc')
sw_se = netCDF4.Dataset(path + 'myd06_radiative_flux_blocking/MYD06_SW_Fluxes_SE.nc')
sw_nw = netCDF4.Dataset(path + 'myd06_radiative_flux_blocking/MYD06_SW_Fluxes_NW.nc')
sw_ne = netCDF4.Dataset(path + 'myd06_radiative_flux_blocking/MYD06_SW_Fluxes_NE.nc')

# LW
lw_sw = netCDF4.Dataset(path + 'myd06_radiative_flux_blocking/MYD06_LW_Fluxes_SW.nc')
lw_se = netCDF4.Dataset(path + 'myd06_radiative_flux_blocking/MYD06_LW_Fluxes_SE.nc')
lw_nw = netCDF4.Dataset(path + 'myd06_radiative_flux_blocking/MYD06_LW_Fluxes_NW.nc')
lw_ne = netCDF4.Dataset(path + 'myd06_radiative_flux_blocking/MYD06_LW_Fluxes_NE.nc')

# Define maximum snowline
snowline_file = netCDF4.Dataset(path + 'sci-adv-products/Monthly_Bare_Ice_2012.nc')
snowline = snowline_file.variables['bare_ice'][1, :, :].filled(np.nan)
max_snowline = (snowline > 0.1)

In [3]:
albedo_current = 0.65
albedo_future_26 = 0.625
albedo_future_45 = 0.584
albedo_future_85 = 0.513

def produce_table(block_file_sw):
    block_cldy = np.nanmean((block_file_sw.variables['cloudiness'][:][max_snowline]) * 100)
    blk_sw_current = np.nanmean((block_file_sw.variables['allsky_sw'][:][max_snowline]) * (1 - albedo_current))
    blk_sw_future_1 = np.nanmean((block_file_sw.variables['allsky_sw'][:][max_snowline]) * (1 - albedo_future_26))
    blk_sw_future_2 = np.nanmean((block_file_sw.variables['allsky_sw'][:][max_snowline]) * (1 - albedo_future_45))
    blk_sw_future_5 = np.nanmean((block_file_sw.variables['allsky_sw'][:][max_snowline]) * (1 - albedo_future_85))
    return block_cldy, blk_sw_current, blk_sw_future_1, blk_sw_future_2, blk_sw_future_5    
    
# Get mean cloudiness
mod_cldy = np.nanmean((np.nanmean(mod.variables['cloudiness'][:], axis=2)[max_snowline]) * 100)

# Get mean radiative flux values
mean_sw_current = np.nanmean((np.nanmean(mod.variables['allsky_sw'][:], axis=2)[max_snowline]) * (1 - albedo_current))
mean_sw_future_1 = np.nanmean((np.nanmean(mod.variables['allsky_sw'][:], axis=2)[max_snowline]) * (1 - albedo_future_26))
mean_sw_future_2 = np.nanmean((np.nanmean(mod.variables['allsky_sw'][:], axis=2)[max_snowline]) * (1 - albedo_future_45))
mean_sw_future_5 = np.nanmean((np.nanmean(mod.variables['allsky_sw'][:], axis=2)[max_snowline]) * (1 - albedo_future_85))
mean_lw_current = np.nanmean(np.nanmean(mod.variables['allsky_lw'][:], axis=2)[max_snowline])

# Get block longwave radiative fluxes
blk_lw_sw_current = lw_sw.variables['allsky_lw'][:][max_snowline]
blk_lw_nw_current = lw_nw.variables['allsky_lw'][:][max_snowline]
blk_lw_se_current = lw_se.variables['allsky_lw'][:][max_snowline]
blk_lw_ne_current = lw_ne.variables['allsky_lw'][:][max_snowline]

# Join LW blocking events
blk_lw_current_all = np.nanmean(np.nanmean((blk_lw_sw_current, blk_lw_nw_current, 
                                            blk_lw_se_current, blk_lw_ne_current), axis=0))

# Get SW blocking events
block_sw = produce_table(sw_sw)
block_nw = produce_table(sw_nw)
block_ne = produce_table(sw_ne)
block_se = produce_table(sw_se)

# Get radiative difference
blk_southwest_current = (block_sw[1] + blk_lw_current_all) - (mean_sw_current + mean_lw_current)
blk_northwest_current = (block_nw[1] + blk_lw_current_all) - (mean_sw_current + mean_lw_current)
blk_northeast_current = (block_ne[1] + blk_lw_current_all) - (mean_sw_current + mean_lw_current)
blk_southeast_current = (block_se[1] + blk_lw_current_all) - (mean_sw_current + mean_lw_current)

blk_southwest_future_1 = (block_sw[2] + blk_lw_current_all) - (mean_sw_future_1 + mean_lw_current)
blk_northwest_future_1 = (block_nw[2] + blk_lw_current_all) - (mean_sw_future_1 + mean_lw_current)
blk_northeast_future_1 = (block_ne[2] + blk_lw_current_all) - (mean_sw_future_1 + mean_lw_current)
blk_southeast_future_1 = (block_se[2] + blk_lw_current_all) - (mean_sw_future_1 + mean_lw_current)

blk_southwest_future_2 = (block_sw[3] + blk_lw_current_all) - (mean_sw_future_2 + mean_lw_current)
blk_northwest_future_2 = (block_nw[3] + blk_lw_current_all) - (mean_sw_future_2 + mean_lw_current)
blk_northeast_future_2 = (block_ne[3] + blk_lw_current_all) - (mean_sw_future_2 + mean_lw_current)
blk_southeast_future_2 = (block_se[3] + blk_lw_current_all) - (mean_sw_future_2 + mean_lw_current)

blk_southwest_future_5 = (block_sw[4] + blk_lw_current_all) - (mean_sw_future_5 + mean_lw_current)
blk_northwest_future_5 = (block_nw[4] + blk_lw_current_all) - (mean_sw_future_5 + mean_lw_current)
blk_northeast_future_5 = (block_ne[4] + blk_lw_current_all) - (mean_sw_future_5 + mean_lw_current)
blk_southeast_future_5 = (block_se[4] + blk_lw_current_all) - (mean_sw_future_5 + mean_lw_current)

cloudiness = list((block_sw[0]-mod_cldy, block_nw[0]-mod_cldy, block_ne[0]-mod_cldy, block_se[0]-mod_cldy))
blk_current = list((blk_southwest_current, blk_northwest_current,
                    blk_northeast_current, blk_southeast_current))
blk_future_1 = list((blk_southwest_future_1, blk_northwest_future_1,
                    blk_northeast_future_1, blk_southeast_future_1))
blk_future_2 = list((blk_southwest_future_2, blk_northwest_future_2,
                    blk_northeast_future_2, blk_southeast_future_2))
blk_future_5 = list((blk_southwest_future_5, blk_northwest_future_5,
                    blk_northeast_future_5, blk_southeast_future_5))

blk_df = pd.DataFrame((cloudiness, blk_current, blk_future_1, blk_future_2, blk_future_5))
blk_df.columns = ['Southwest', 'Northwest', 'Northeast', 'Southeast']

/var/folders/xj/5ps5mr8d5ysbd2mxxqjg3k800000gq/T/ipykernel_67908/1144810055.py:15: RuntimeWarning: Mean of empty slice
  mod_cldy = np.nanmean((np.nanmean(mod.variables['cloudiness'][:], axis=2)[max_snowline]) * 100)
/var/folders/xj/5ps5mr8d5ysbd2mxxqjg3k800000gq/T/ipykernel_67908/1144810055.py:18: RuntimeWarning: Mean of empty slice
  mean_sw_current = np.nanmean((np.nanmean(mod.variables['allsky_sw'][:], axis=2)[max_snowline]) * (1 - albedo_current))
/var/folders/xj/5ps5mr8d5ysbd2mxxqjg3k800000gq/T/ipykernel_67908/1144810055.py:19: RuntimeWarning: Mean of empty slice
  mean_sw_future_1 = np.nanmean((np.nanmean(mod.variables['allsky_sw'][:], axis=2)[max_snowline]) * (1 - albedo_future_26))
/var/folders/xj/5ps5mr8d5ysbd2mxxqjg3k800000gq/T/ipykernel_67908/1144810055.py:20: RuntimeWarning: Mean of empty slice
  mean_sw_future_2 = np.nanmean((np.nanmean(mod.variables['allsky_sw'][:], axis=2)[max_snowline]) * (1 - albedo_future_45))
/var/folders/xj/5ps5mr8d5ysbd2mxxqjg3k800000gq/T/ipykerne

In [4]:
blk_df

,Southwest,Northwest,Northeast,Southeast
0,-2.764959,-5.736597,-6.353491,-0.200336
1,3.993805,11.494385,4.944977,-1.620026
2,4.231934,12.268311,5.251099,-1.782837
3,4.622559,13.537476,5.753113,-2.049866
4,5.298889,15.735352,6.622345,-2.512329


In [6]:
(6.6+15.7+5.3-2.5) / 4

6.2749999999999995

In [7]:
(4.0+11.5+5.0-1.6)/4

4.725

In [8]:
(6.27 - 4.725) / 4.5

0.3433333333333333

In [ ]:
+15.7 ± 4.3, +6.6 ± 1.8, and +5.3 ± 1.4 W m-2 

In [9]:
15.7*.27

4.239

In [10]:
6.6*.27

1.782